In [66]:
DEBUG = False
from sys import maxsize
from json import dumps

d = lambda thing: print(dumps(thing, indent=4))
from collections import defaultdict
sample = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""" 

turtle_dict = lambda: defaultdict(turtle_dict)
DEBUG2 = False
def crawl_dir(data):
    result = defaultdict(turtle_dict)
    steps = iter(data.splitlines())
    first_step = next(steps)

    totals = []
    def add_total(tree, parents="TOP", depth=0):
        pad = depth * " "
        depth += 1
        nonlocal totals
        total = 0
        for name, size_or_dir in tree.items():
            if not parents == "TOP":
                parent = '/'.join((parents,name))
            else:
                parent = ""
            DEBUG2 and print (pad + f"{name=}, {size_or_dir=}")
            if isinstance(size_or_dir, int):
                total += size_or_dir
            else:
                dir_total = add_total(size_or_dir, parent, depth)
                total+=dir_total
                DEBUG2 and print (pad + f'{dir_total} received back from {name}')
                totals.append(dir_total)
        return total


    def gather(result, name, depth):
        pad = depth * " "
        depth += 1
        while True:
            try:
                step = next(steps)
            except StopIteration:
                break

            DEBUG and print (pad + f'{name=} {step=}')
            if step.startswith("$ cd") and step.endswith(".."):
                return
            elif step.startswith("$ cd"):
                dir = step.split()[2]
                DEBUG and print (pad + f"{dir}")
                gather(result[name], dir, depth)
            elif step.startswith("$ ls"):
                DEBUG and print (pad + "ls")
            elif step.startswith("dir"):
                pass
                # result[step.split()[1]]
            elif step[0].isdigit():
                parts = step.split()
                DEBUG and print (pad + f"{name=} {parts=}")
                size = int(parts[0])
                result[name][parts[1]] = size
            else:
                DEBUG and print (pad + "what")
                break


    gather(result, first_step.split()[2], 0)

    DEBUG and d(result)

    add_total(result)
    DEBUG and d(totals)
    print(sum([t for t in totals if t < 100000]))
    
    total = max(totals)
    difference = 30000000 - (70000000 - total)
    best_answer = maxsize
    for t in totals:
        if t > difference and t < best_answer:
            best_answer = t
    print(best_answer)
    



DEBUG = False
crawl_dir(sample)

95437
24933642


In [67]:
crawl_dir(open('d7.input').read())

1206825
9608311
